In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder

plt.rc('font', family='AppleGothic') # 한글출력
plt.rcParams['axes.unicode_minus'] = False # 축 -표시

In [3]:
train = pd.read_csv('./PJT002_train.csv')
test = pd.read_csv('./PJT002_test.csv')
valid = pd.read_csv('./PJT002_validation.csv')
# 유일하게 gas 사용량 음수인 행을 양수로 변경
train['gas_engry_us_201507'] = np.abs(train['gas_engry_us_201507'])

C:\Users\samsung\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# 타겟 변수 int형으로 변경
train['fr_yn'] = np.where(train['fr_yn']=='Y', 1 ,0) 
valid['fr_yn'] = np.where(valid['fr_yn']=='Y', 1 ,0)

In [5]:
# 강수량 결측은 0으로 변경
train['prcpttn'] = train['prcpttn'].fillna(0) 
valid['prcpttn'] = valid['prcpttn'].fillna(0) 
test['prcpttn'] = test['prcpttn'].fillna(0) 

In [6]:
# 향후 변수 생성을 위해 시간 관련 변수들 생성
train['dt_of_fr'] = pd.to_datetime(train['dt_of_fr'])
train['year'] = train['dt_of_fr'].dt.year
train['month'] = train['dt_of_fr'].dt.month
train['hour'] = train['dt_of_fr'].dt.hour
train['weekday'] = train['dt_of_fr'].dt.weekday


valid['dt_of_fr'] = pd.to_datetime(valid['dt_of_fr'])
valid['year'] = valid['dt_of_fr'].dt.year
valid['month'] = valid['dt_of_fr'].dt.month
valid['hour'] = valid['dt_of_fr'].dt.hour
valid['weekday'] = valid['dt_of_fr'].dt.weekday

test['dt_of_fr'] = pd.to_datetime(test['dt_of_fr'])
test['year'] = test['dt_of_fr'].dt.year
test['month'] = test['dt_of_fr'].dt.month
test['hour'] = test['dt_of_fr'].dt.hour
test['weekday'] = test['dt_of_fr'].dt.weekday

In [7]:
# 건축연도 변수를 건물연식(오래된 정도, 기간)으로 즉 연속형으로 변경
train['dt_of_athrztn'] = train['dt_of_athrztn'].fillna(9999) # 결측치 제거
train['dt_of_athrztn'] = train['dt_of_athrztn'].astype('str').str.slice(stop=4).astype('int') # 연도만 추출
train['dt_of_athrztn'] = np.where(train['dt_of_athrztn'].astype('str').str.slice(stop=1).astype('int')>1, train['dt_of_athrztn'].astype('str').str.slice(stop=2).astype('int')+1900, train['dt_of_athrztn']) # 4자리 연도 아닌 경우 처리
train['dt_of_athrztn'] = 2019 - train['dt_of_athrztn'] # 오래된 정도로 변경
train['dt_of_athrztn'] = np.where(train['dt_of_athrztn']==-7980, np.nan, train['dt_of_athrztn']) # 결측 처리

valid['dt_of_athrztn'] = valid['dt_of_athrztn'].fillna(9999) # 결측치 제거
valid['dt_of_athrztn'] = valid['dt_of_athrztn'].astype('str').str.slice(stop=4).astype('int') # 연도만 추출
valid['dt_of_athrztn'] = np.where(valid['dt_of_athrztn'].astype('str').str.slice(stop=1).astype('int')>1, valid['dt_of_athrztn'].astype('str').str.slice(stop=2).astype('int')+1900, valid['dt_of_athrztn']) # 4자리 연도 아닌 경우 처리
valid['dt_of_athrztn'] = 2019 - valid['dt_of_athrztn'] # 오래된 정도로 변경
valid['dt_of_athrztn'] = np.where(valid['dt_of_athrztn']==-7980, np.nan, valid['dt_of_athrztn']) # 결측 처리

test['dt_of_athrztn'] = test['dt_of_athrztn'].fillna(9999) # 결측치 제거
test['dt_of_athrztn'] = test['dt_of_athrztn'].astype('str').str.slice(stop=4).astype('int') # 연도만 추출
test['dt_of_athrztn'] = np.where(test['dt_of_athrztn'].astype('str').str.slice(stop=1).astype('int')>1, test['dt_of_athrztn'].astype('str').str.slice(stop=2).astype('int')+1900, test['dt_of_athrztn']) # 4자리 연도 아닌 경우 처리
test['dt_of_athrztn'] = 2019 - test['dt_of_athrztn'] # 오래된 정도로 변경
test['dt_of_athrztn'] = np.where(test['dt_of_athrztn']==-7980, np.nan, test['dt_of_athrztn']) # 결측 처리

In [8]:
# emd_nm의 행정단위만 추출해서 변수 생성
# emd_nm2: 시, 군 단위
# emd_nm3: 읍,면,동 단위
train['emd_nm2'] = train['emd_nm'].str.slice(start=5, stop=8).str.slice(start=-1)
train['emd_nm3'] = train['emd_nm'].str.slice(start=-1)
train['emd_nm3'] = np.where(train['emd_nm3']=='가', '동', train['emd_nm3']) # ~동 ~가는 동으로 변경

valid['emd_nm2'] = valid['emd_nm'].str.slice(start=5, stop=8).str.slice(start=-1)
valid['emd_nm3'] = valid['emd_nm'].str.slice(start=-1)
test['emd_nm2'] = test['emd_nm'].str.slice(start=5, stop=8).str.slice(start=-1)
test['emd_nm3'] = test['emd_nm'].str.slice(start=-1)

del train['emd_nm']
del valid['emd_nm']
del test['emd_nm']

train = train.drop(['fr_fghtng_fclt_spcl_css_5_yn',
                    'slf_fr_brgd_yn','blk_dngrs_thng_mnfctr_yn','fr_mn_cnt'], axis=1)
valid = valid.drop(['fr_fghtng_fclt_spcl_css_5_yn','slf_fr_brgd_yn',
                    'blk_dngrs_thng_mnfctr_yn','fr_mn_cnt'], axis=1)
test = test.drop(['fr_fghtng_fclt_spcl_css_5_yn','slf_fr_brgd_yn',
                  'blk_dngrs_thng_mnfctr_yn','fr_mn_cnt'], axis=1)

In [9]:
# 가스,전기 컬럼명 리스트
gas_col =[]
ele_col = []
for year in ['2014', '2015','2016','2017','2018']:
  for month in range(1,13):
    if month <10:
      month = '0' + str(month)
      period = year + month
      gas = 'gas_engry_us_' + period
      ele = 'ele_engry_us_' + period
      gas_col.append(gas)
      ele_col.append(ele)
    else:
      month = str(month)
      period = year + month
      gas = 'gas_engry_us_' + period
      ele = 'ele_engry_us_' + period
      gas_col.append(gas)
      ele_col.append(ele)

In [10]:
# 에너지 결측
train['ele_null']= train[ele_col].isna().sum(axis=1)
train['gas_null']= train[gas_col].isna().sum(axis=1)
valid['ele_null']= valid[ele_col].isna().sum(axis=1)
valid['gas_null']= valid[gas_col].isna().sum(axis=1)
test['ele_null']= test[ele_col].isna().sum(axis=1)
test['gas_null']= test[gas_col].isna().sum(axis=1)

In [11]:
# 가스, 전기 관련 변수의 대푯값 변수 생성
def energy_mean(data):
    for i in ['01','02','03','04','05','06','07','08','09','10','11','12']:
        ele1 = 'ele_engry_us_' + '2014'+ i
        ele2 = 'ele_engry_us_' + '2015'+i
        ele3 = 'ele_engry_us_' + '2016'+i
        ele4 = 'ele_engry_us_' + '2017'+i
        ele5 ='ele_engry_us_' + '2018'+ i
        
        gas1 = 'gas_engry_us_' + '2014'+i
        gas2 = 'gas_engry_us_' + '2015'+i
        gas3 = 'gas_engry_us_' + '2016'+i
        gas4 = 'gas_engry_us_' + '2017'+i
        gas5 = 'gas_engry_us_' + '2018'+i

        g_mean = 'g_mean' + '_' + i
        e_mean = 'e_mean' + '_' + i
        g_median = 'g_median' + '_' + i
        e_median = 'e_median' + '_' + i
        g_std = 'g_std' + '_' + i
        e_std = 'e_std' + '_' + i
        g_max = 'g_max' + '_' + i
        e_max = 'e_max' + '_' + i
        g_min = 'g_min' + '_' + i
        e_min = 'e_min' + '_' + i
        
        data[g_mean] = data[[gas1,gas2,gas3,gas4,gas5]].mean(axis=1)
        data[e_mean] = data[[ele1,ele2,ele3,ele4,ele5]].mean(axis=1)
        data[g_median] =data[[gas1,gas2,gas3,gas4,gas5]].median(axis=1)
        data[e_median] = data[[ele1,ele2,ele3,ele4,ele5]].median(axis=1) 
        data[g_std] = data[[gas1,gas2,gas3,gas4,gas5]].std(axis=1)
        data[e_std] = data[[ele1,ele2,ele3,ele4,ele5]].std(axis=1)
        data[g_max] = data[[gas1,gas2,gas3,gas4,gas5]].max(axis=1)
        data[e_max] = data[[ele1,ele2,ele3,ele4,ele5]].max(axis=1)
        data[g_min] = data[[gas1,gas2,gas3,gas4,gas5]].min(axis=1)
        data[e_min] = data[[ele1,ele2,ele3,ele4,ele5]].min(axis=1)
    return data

In [12]:
train = energy_mean(train)
valid = energy_mean(valid)
test = energy_mean(test)

In [13]:
# 전기, 가스 사용량의 상한-하한
def energy_variate(data):
    for i in ['01','02','03','04','05','06','07','08','09','10','11','12']:
        ele1 = 'ele_engry_us_' + '2014'+ i
        ele2 = 'ele_engry_us_' + '2015'+i
        ele3 = 'ele_engry_us_' + '2016'+i
        ele4 = 'ele_engry_us_' + '2017'+i
        ele5 ='ele_engry_us_' + '2018'+ i
        
        gas1 = 'gas_engry_us_' + '2014'+i
        gas2 = 'gas_engry_us_' + '2015'+i
        gas3 = 'gas_engry_us_' + '2016'+i
        gas4 = 'gas_engry_us_' + '2017'+i
        gas5 = 'gas_engry_us_' + '2018'+i
        
        g_mean = 'g_mean' + '_' + i
        e_mean = 'e_mean' + '_' + i
        
        data[g_mean] = data[[gas1,gas2,gas3,gas4,gas5]].mean(axis=1)
        data[e_mean] = data[[ele1,ele2,ele3,ele4,ele5]].mean(axis=1)
        
        data['g_va'] = (data[[gas1,gas2,gas3,gas4,gas5]].max(axis=1) - data[[gas1,gas2,gas3,gas4,gas5]].min(axis=1)) /\
        data[[gas1,gas2,gas3,gas4,gas5]].max(axis=1)
        
        data['e_va'] = (data[[ele1,ele2,ele3,ele4,ele5]].max(axis=1)- data[[ele1,ele2,ele3,ele4,ele5]].min(axis=1))/\
        data[[ele1,ele2,ele3,ele4,ele5]].max(axis=1)

    return data

In [14]:
train = energy_variate(train)
valid = energy_variate(valid)
test = energy_variate(test)

In [15]:
from time import time
import datetime

# 화재 난 날 에너지 사용, 화재 난 날 - 평년 에너지 사용

def dday_energy(data):
    data['month'] = np.where(data['month'].astype('int')<10, '0'+data['month'].astype('str'), data['month'].astype('str'))
    data['gas_date'] = 'gas_engry_us_' + data['year'].astype('str') + data['month']
    data['ele_date'] = 'ele_engry_us_' + data['year'].astype('str') + data['month']
    data['gas_dday'] = 0
    data['ele_dday'] = 0
    data['ele_diff'] = 0
    data['gas_diff'] = 0
    
    for col in ele_col:
        data.loc[data['ele_date']== col,'ele_dday']= \
        data[data['ele_date']== col][col]
        
    for col in data['month'].unique():
        data.loc[data['month']== col,'ele_diff']= \
        data['ele_dday'] - data['e_mean_' + col]
        
    for col in gas_col:
        data.loc[data['gas_date']== col,'gas_dday']= \
        data[data['gas_date']== col][col]
        
    for col in data['month'].unique():
        data.loc[data['month']== col,'gas_diff']= \
        data['gas_dday'] - data['g_mean_' + col]

    return data

In [16]:
train = dday_energy(train)
valid = dday_energy(valid)
test = dday_energy(test)

In [17]:
# 건물연면적을 지상,지하로 나눔
def flr_ar(data):
    data['grnd_ar'] = data['bldng_ar'] * data['ttl_grnd_flr']
    data['dwn_ar'] = data['bldng_ar'] * data['ttl_dwn_flr']
    return data

#def ar_per_bldng(data):
#    data['bldng_1_ar'] = data['bldng_ar'] / data['bldng_cnt']
#    return data

In [18]:
train = flr_ar(train)
valid = flr_ar(valid)
test = flr_ar(test)

In [19]:
# 필요 없어진 컬럼 제거
train = train.drop(gas_col + ele_col, axis=1)
valid = valid.drop(gas_col + ele_col, axis=1)
test = test.drop(gas_col + ele_col, axis=1)

train = train.drop(['ele_date','gas_date'], axis=1)
valid = valid.drop(['ele_date','gas_date'], axis=1)
test = test.drop(['ele_date','gas_date'], axis=1)

In [ ]:
train.to_csv('train_pre.csv', index=False)
valid.to_csv('valid_pre.csv', index=False)
test.to_csv('test_pre.csv', index=False)